In [8]:
load_items = """LOAD CSV WITH HEADERS FROM 'file:///Groceries_dataset.csv' AS line
MERGE (member:Member {number: toInteger(line.Member_number)})
MERGE (item:Item {description: line.itemDescription})"""

In [7]:
relationships = """LOAD CSV WITH HEADERS FROM 'file:///Groceries_dataset.csv' AS line
MATCH (member:Member {number: toInteger(line.Member_number)})
MATCH (item:Item {description: line.itemDescription})
MERGE (member)-[:PURCHASED {on: line.Date}]->(item)"""

In [1]:
from neo4j import GraphDatabase

/Users/pratistha99/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))

In [3]:

def get_recommendations(tx, member_number):
    query = (
        """
        MATCH (targetMember:Member {number: $memberNumber})-[:PURCHASED]->(item:Item)
        WITH targetMember, item
        MATCH (item)<-[p:PURCHASED]-(similarMember:Member)-[p2:PURCHASED]->(recommendation:Item)
        WHERE NOT (targetMember)-[:PURCHASED]->(recommendation) AND targetMember <> similarMember
        WITH recommendation, COUNT(p2) AS purchases, COLLECT(similarMember) AS similarMembers
        ORDER BY purchases DESC
        WITH recommendation, purchases, similarMembers
        WHERE SIZE(similarMembers) > 1
        RETURN recommendation.description AS RecommendedItem, purchases, similarMembers
        LIMIT 5
        """
    )
    results = tx.run(query, memberNumber=member_number)
    return [(record["RecommendedItem"], record["purchases"]) for record in results]



In [4]:

with driver.session() as session:
    recommendations = session.read_transaction(get_recommendations, 1808)
    for item, purchases in recommendations:
        print(f"Item: {item}, Purchases: {purchases}")

driver.close()


/var/folders/22/q3sbq4cn5qsf37t1lqjztgvm0000gn/T/ipykernel_5713/3501515833.py:2: DeprecationWarning: read_transaction has been renamed to execute_read
  recommendations = session.read_transaction(get_recommendations, 1808)


Item: other vegetables, Purchases: 4037
Item: soda, Purchases: 3063
Item: yogurt, Purchases: 2852
Item: root vegetables, Purchases: 2229
Item: sausage, Purchases: 2044
